In [1]:
import requests
import json
import os
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
class Weather_API:
    
    def __init__ (self, keyword):
        self.keyword = keyword
        
    def json_print(self, obj):
        # create a formatted string of the Python JSON object
        with open('api_data.txt', 'w') as json_file:
            json.dump(obj, json_file)
        text = json.dumps(obj, sort_keys=True, indent=4)
        print(text)
        
    def create_dataframe(self, obj):
        
        # creating a dataframe from nested JSON objects
        FIELDS = ["source.id", "source.name", "author", "title", "description", "url", "urlToImage", "publishedAt", "content"]
        df = pd.json_normalize(obj['articles'])
        final_df = df[FIELDS]
        #final_df.set_index('source.id', inplace = True)
        display(final_df.head())

    
    def news_api(self):
        
        # Use the news-api to obtain articles published from
        url = ('https://newsapi.org/v2/everything?'
       'q={keyword}&'
       'apiKey=4e70cabb80884db08524a28ac33cdc1d'.format(keyword = self.keyword))
        
        
        response = requests.get(url)
        if (response.status_code == 200):
            print('API call successful!')
            json_response = response.json()
            if(len(json_response['articles']) == 0):
                print('No News Articles Found')
            else:
                
                # Print a String in Json Format
                self.json_print(json_response)
                
                
                
                # Create a pandas DataFrame
                self.create_dataframe(json_response)
                    
        else:
            print('Status code: ', response.status_code)

In [3]:
class Web_Scraping:
    
    def __init__(self, location):
        self.location = location
        
    def selenium_webdriver(self):
        
        # Start the Driver
        driver = webdriver.Chrome(executable_path = r"C:\Users\Aditya\Downloads\chromedriver_win32\chromedriver.exe")
        
        # Hit the url of NASA Earth Data website and wait for 15 seconds.
        url = ('https://earthdata.nasa.gov/search?q={location}'.format(location = self.location))
        driver.get(url)
        time.sleep(15)
        
        # Driver scrolls down 25 times to load the table.
        for i in range(0,30):
            driver.execute_script("window.scrollBy(0,6000)")
            time.sleep(10)
            
        # Fetch the webpage and store in a variable.
        webpage = driver.page_source
        
        # Parse the page using BeautifulSoup
        HTMLPage = BeautifulSoup(webpage, 'html.parser')
        
        titles = []
        description = []
        links = []

        for lists in HTMLPage.find_all(class_ = 'result'):
            if (lists.span.text != '' and len(lists.find_all('p')) != 0):
                titles.append(lists.span.text)
                description.append(lists.find('p', class_ = '').text)
                links.append(lists.find('p', class_ = 'search-link').text)
        
        # Create a DataFrame
        df = pd.DataFrame(list(zip(titles, description, links)),
               columns =['title', 'description', 'link'])
        
        display(df)
        
        # Store to csv file
        df.to_csv('ws.csv', sep=',', index=False,header=True)
        
        print('Web Scraping Successful!')

In [ ]:
keyword = input('Enter Keyword to be searched: ').lower()
w_api = Weather_API(keyword)
w_api.news_api()

location = input('Enter Location: ').lower()
ws = Web_Scraping('India')
ws.selenium_webdriver()

Enter Keyword to be searched:  mumbai


API call successful!
{
    "articles": [
        {
            "author": "Shubhangi Goel,Meghan Morris,Jyoti Mann",
            "content": "Anant Ambani with his parents Mukesh and Nita.PUNIT PARANJPEPUNIT PARANJPE/ Getty Images\r\n<ul><li>Radhika Merchant and Anant Ambani's wedding begins on Friday.</li><li>The three-day gathering will fe\u2026 [+3850 chars]",
            "description": "Radhika Merchant and Anant Ambani's wedding in Mumbai this weekend is set to be a star-studded affair.",
            "publishedAt": "2024-07-12T13:10:17Z",
            "source": {
                "id": "business-insider",
                "name": "Business Insider"
            },
            "title": "Meet the rich and famous people in Mumbai for the Ambani wedding",
            "url": "https://www.businessinsider.com/ambani-wedding-guests-celebrities-rich-powerful-famous-people-invited-2024-7",
            "urlToImage": "https://i.insider.com/66911e5e8893442a51560068?width=1200&format=jpeg"
        },

,source.id,source.name,author,title,description,url,urlToImage,publishedAt,content
0,business-insider,Business Insider,"Shubhangi Goel,Meghan Morris,Jyoti Mann",Meet the rich and famous people in Mumbai for ...,Radhika Merchant and Anant Ambani's wedding in...,https://www.businessinsider.com/ambani-wedding...,https://i.insider.com/66911e5e8893442a51560068...,2024-07-12T13:10:17Z,Anant Ambani with his parents Mukesh and Nita....
1,business-insider,Business Insider,Nora Redmond,Heavy rain could put a dampener on the lavish ...,Over half the flights arriving in Mumbai on Fr...,https://www.businessinsider.com/ambani-wedding...,https://i.insider.com/66910e7e5439fb59cebcc5d9...,2024-07-12T16:33:13Z,The entrance to the Ambani family's Mumbai hom...
2,business-insider,Business Insider,Pete Syme,I was on a flight that received a bomb threat....,Molina Singh's Akasa Air flight was diverted a...,https://www.businessinsider.com/i-was-on-bomb-...,https://i.insider.com/6698e806fb2b6bedb0583474...,2024-07-19T08:00:02Z,Passengers lined up to board an Akasa Air flig...
3,business-insider,Business Insider,Maria Noyen,Inside the Ambani wedding: Details you may hav...,The Ambani wedding culminated in a three-day e...,https://www.businessinsider.com/ambani-wedding...,https://i.insider.com/669575f08893442a5156bb98...,2024-07-15T22:02:20Z,Anant Ambani and Radhika Merchant on July 5 at...
4,None,Design-milk.com,Vy Yang,Sony Music’s New Hub in Mumbai Hits All the Ri...,M Moser Associates marries Mumbai's heritage w...,https://design-milk.com/sony-musics-new-hub-in...,https://design-milk.com/images/2024/07/m-moser...,2024-07-10T16:00:00Z,"In Mumbai, a modern office transformation is s..."
